In [24]:
import sklearn
print(sklearn.__version__)


1.6.0


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,make_column_selector

In [26]:
d=pd.read_csv('C:/Users\ESHITA\Downloads/accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv')

C:\Users\ESHITA\AppData\Local\Temp\ipykernel_22324\3225256400.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  d=pd.read_csv('C:/Users\ESHITA\Downloads/accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv')


In [27]:
data=d[["loan_amnt","term","int_rate","fico_range_low","annual_inc","dti","emp_length","purpose","open_acc","total_acc","revol_util",
"inq_last_6mths","loan_status"]]
data=data[data["loan_status"].isin(["Fully Paid","Charged Off"])]
print(data.head(20))

    loan_amnt        term  int_rate  fico_range_low  annual_inc    dti  \
0      3600.0   36 months     13.99           675.0     55000.0   5.91   
1     24700.0   36 months     11.99           715.0     65000.0  16.06   
2     20000.0   60 months     10.78           695.0     63000.0  10.78   
4     10400.0   60 months     22.45           695.0    104433.0  25.37   
5     11950.0   36 months     13.44           690.0     34000.0  10.20   
6     20000.0   36 months      9.17           680.0    180000.0  14.67   
7     20000.0   36 months      8.49           705.0     85000.0  17.61   
8     10000.0   36 months      6.49           685.0     85000.0  13.07   
9      8000.0   36 months     11.48           700.0     42000.0  34.80   
12     1400.0   36 months     12.88           700.0     64000.0  34.95   
13    18000.0   60 months     19.48           665.0    150000.0   9.39   
14    28000.0   36 months      6.49           720.0     92000.0  21.60   
15     9600.0   36 months      7.49   

In [28]:
data["purpose"].unique()

array(['debt_consolidation', 'small_business', 'home_improvement',
       'major_purchase', 'credit_card', 'other', 'house', 'vacation',
       'car', 'medical', 'moving', 'renewable_energy', 'wedding',
       'educational'], dtype=object)

In [41]:
data["term"].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [40]:
X=data.drop('loan_status',axis=1)
y=data['loan_status']

In [30]:
num_pipe=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
cat_pipe=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore',
                            sparse_output=False,
                            dtype=int))
])
ct=ColumnTransformer( transformers=[
        ('num', num_pipe,
         make_column_selector(dtype_include=(int, float))),
        ('cat', cat_pipe,
         make_column_selector(dtype_include=["object", "string"]))
    ])

In [31]:
X_new=ct.fit_transform(X)


In [32]:
type(y)

pandas.core.series.Series

In [33]:

y_new= y.map({
    "Fully Paid": 0,
    "Charged Off": 1
})

In [34]:
X_train,X_test,y_train,y_test=train_test_split(X_new,y_new,test_size=0.2,random_state=42)

In [35]:
xgb=XGBClassifier(
    n_estimators=120,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
)
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None,
              num_parallel_tree=None, ...)

In [36]:
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score
y_prob=xgb.predict_proba(X_test)
y_pred=(y_prob[:,1]>0.4).astype(int)
print(classification_report(y_test,y_pred))
roc_auc_score(y_test, y_prob[:,1])

              precision    recall  f1-score   support

           0       0.82      0.95      0.88    215608
           1       0.47      0.18      0.26     53454

    accuracy                           0.80    269062
   macro avg       0.65      0.57      0.57    269062
weighted avg       0.75      0.80      0.76    269062



0.7124891181689674

In [37]:
train_acc = accuracy_score(y_train, xgb.predict(X_train))
test_acc  = accuracy_score(y_test, xgb.predict(X_test))
print(train_acc)
print(test_acc)

0.802816822888405
0.8037589849179743


In [38]:
y_train.value_counts(normalize=True)


loan_status
0    0.800134
1    0.199866
Name: proportion, dtype: float64

In [39]:
import pickle
final_pipeline = Pipeline(steps=[
    ("preprocessing", ct),
    ("model", xgb)
])

with open("pre_screening.pkl", "wb") as f:
    pickle.dump(final_pipeline, f)



The XGBoost classifier achieved the best performance among all models, demonstrating strong generalization with nearly identical training and test scores. The model significantly outperformed the baseline, indicating that it successfully learned meaningful patterns from the data without overfitting. ROC-AUC was used as the primary evaluation metric, making the model suitable for credit risk prediction where ranking defaulters is critical.